In [125]:
# Import packages
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import re
from shapely.geometry import Polygon, MultiPolygon
from rasterstats import zonal_stats
from osgeo import gdal
import rasterio
import rasterstats
import time
sys.path.append(os.path.abspath("C:/Users/mieke/Documents/Msc Thesis/Notebooks Python/Final/"))
import functions_model #import python file containing the functions

In [ ]:
# For updates
import importlib
importlib.reload(functions_model)

In [2]:
# Create a shapeName column containing the names of each Insurance Unit in lowercase and stripped
insurance_names = gpd.read_file(r'C:\Users\mieke\Documents\Msc Thesis\Notebooks Python\Final\insurance_names.geojson')
insurance_names['shapeName'] = insurance_names['Insurance Unit'].copy()
insurance_names.shapeName = insurance_names.shapeName.str.lower()
insurance_names.shapeName = insurance_names.shapeName.str.strip()

In [3]:
# Import data
yields = gpd.read_file(r'C:\Users\mieke\Documents\Msc Thesis\Notebooks Python\Final\yields_geometry.geojson') # All yield observations for paddy, maize and soghrum
obs_calculation = gpd.read_file(r'C:\Users\mieke\Documents\Msc Thesis\Notebooks Python\Final\obs_calculation.geojson') # All unique geometries for each crop type per year, season, unit

In [4]:
obs_paddy = obs_calculation[obs_calculation['Crop'] == 'paddy'] # 7285
obs_maize = obs_calculation[obs_calculation['Crop'] == 'maize (makka)'] # 6008
obs_sorghum = obs_calculation[obs_calculation['Crop'] == 'sorghum (jowar/great millet)'] # 4788

yields_paddy = yields[yields['Crop'] == 'paddy'] # 8461
yields_maize = yields[yields['Crop'] == 'maize (makka)'] # 7480 
yields_sorghum = yields[yields['Crop'] == 'sorghum (jowar/great millet)'] # 5411

In [5]:
yields_copy = yields.copy()
obs_calculation_copy = obs_calculation.copy()
# yields = yields_copy.copy()
# obs_calculation = obs_calculation_copy.copy()

In [ ]:
print(np.size(yields_maize[yields_maize['Season'] == 'Rabi'],0)) # 819
print(np.size(yields_maize[yields_maize['Season'] == 'Summer'],0)) # 51
print(np.size(yields_maize[yields_maize['Season'] == 'Kharif'],0), '\n') # 6610

print(np.size(yields_sorghum[yields_sorghum['Season'] == 'Rabi'],0)) # 4636
print(np.size(yields_sorghum[yields_sorghum['Season'] == 'Summer'],0)) # 8
print(np.size(yields_sorghum[yields_sorghum['Season'] == 'Kharif'],0), '\n') # 767

In [7]:
# Create dataframes per season per year such that we can calculate statistics based on this information (e.g. parameter value differs per year, season)
maize_2016_rabi = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Rabi')] # 241
maize_2016_zaid = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Summer')] # 23
maize_2016_kharif = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Kharif')] # 1916
maize_2017_rabi = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Rabi')] # 251
maize_2017_zaid = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Summer')] # 16
maize_2017_kharif = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Kharif')] # 1760
maize_2018_rabi = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Rabi')] # 225
maize_2018_zaid = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Summer')] # 8
maize_2018_kharif = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Kharif')] # 1568

sorghum_2016_rabi = obs_sorghum[(obs_sorghum['Year'] == 2016) & (obs_sorghum['Season'] == 'Rabi')] # 1168
sorghum_2016_zaid = obs_sorghum[(obs_sorghum['Year'] == 2016) & (obs_sorghum['Season'] == 'Summer')] # 4
sorghum_2016_kharif = obs_sorghum[(obs_sorghum['Year'] == 2016) & (obs_sorghum['Season'] == 'Kharif')] # 207
sorghum_2017_rabi = obs_sorghum[(obs_sorghum['Year'] == 2017) & (obs_sorghum['Season'] == 'Rabi')] # 1632
sorghum_2017_zaid = obs_sorghum[(obs_sorghum['Year'] == 2017) & (obs_sorghum['Season'] == 'Summer')] # 1
sorghum_2017_kharif = obs_sorghum[(obs_sorghum['Year'] == 2017) & (obs_sorghum['Season'] == 'Kharif')] # 163
sorghum_2018_rabi = obs_sorghum[(obs_sorghum['Year'] == 2018) & (obs_sorghum['Season'] == 'Rabi')] # 1568
sorghum_2018_zaid = obs_sorghum[(obs_sorghum['Year'] == 2018) & (obs_sorghum['Season'] == 'Summer')] # 0
sorghum_2018_kharif = obs_sorghum[(obs_sorghum['Year'] == 2018) & (obs_sorghum['Season'] == 'Kharif')] # 134

In [ ]:
print(np.size(maize_2016_rabi,0)) # 184
print(np.size(maize_2016_zaid,0)) # 210
print(np.size(maize_2016_kharif,0))# 2098
print(np.size(maize_2017_rabi,0)) # 222
print(np.size(maize_2017_zaid,0)) # 380
print(np.size(maize_2017_kharif,0)) # 1739
print(np.size(maize_2018_rabi,0)) # 192
print(np.size(maize_2018_zaid,0)) # 387
print(np.size(maize_2018_kharif,0), '\n') # 1873

print(np.size(sorghum_2016_rabi,0)) # 184
print(np.size(sorghum_2016_zaid,0)) # 210
print(np.size(sorghum_2016_kharif,0)) # 2098
print(np.size(sorghum_2017_rabi,0)) # 222
print(np.size(sorghum_2017_zaid,0)) # 380
print(np.size(sorghum_2017_kharif,0)) # 1739
print(np.size(sorghum_2018_rabi,0)) # 192
print(np.size(sorghum_2018_zaid,0)) # 387
print(np.size(sorghum_2018_kharif,0)) # 1873

In [11]:
# This cell creates monthly averages for each parameter of interest and stores them in folder init_path
beginyear = 2015
endyear = 2018
paths = ['D:/Data_download/GEE/NDVI/Karnataka/cropmask/', 'D:/Data_download/GEE/EVI/Karnataka/cropmask/', 'D:/Data_download/Copernicus/FAPAR/Karnataka/cropmask/', 'D:/Data_download/Copernicus/LAI/Karnataka/cropmask/', 'D:/Data_download/Other/SIF/Karnataka/cropmask/', 'D:/Data_download/GEE/SSM/Karnataka/cropmask/', 'D:/Data_download/GEE/SUSM/Karnataka/cropmask/', 'D:/Data_download/GEE/LST/Karnataka/cropmask/', 'D:/Data_download/GEE/LSTN/Karnataka/cropmask/', 'D:/Data_download/GEE/Precipitation/Karnataka/cropmask/']
init_path = 'D:/monthly_images/crop/'
parameters = ['NDVI', 'EVI', 'FAPAR', 'LAI', 'SIF', 'SSM', 'SUSM', 'LST', 'LSTN', 'Precipitation']
for i in range(len(parameters)):
    dest_path = init_path + parameters[i] + '/'
    # if i == 'Precipitation':
    #     monthly_accumulates(paths[i], dest_path, beginyear, endyear)
    # else:
    functions_model.monthly_averages(paths[i], dest_path, beginyear, endyear)

In [14]:
# This cell sets values for the parameters we use for function time_observations within the next cell
paths = ['D:/monthly_images/crop/NDVI/', 'D:/monthly_images/crop/EVI/', 'D:/monthly_images/crop/FAPAR/', 'D:/monthly_images/crop/LAI/', 'D:/monthly_images/crop/SIF/', 'D:/monthly_images/crop/SSM/', 'D:/monthly_images/crop/SUSM/', 'D:/monthly_images/crop/LST/', 'D:/monthly_images/crop/LSTN/', 'D:/monthly_images/crop/Precipitation/']
methods = [False, False, False, False, True, True, True, False, False, True]
statistic = ['median']
parameters = ['NDVI', 'EVI', 'FAPAR', 'LAI', 'SIF', 'SSM', 'SUSM', 'LST', 'LSTN', 'Precipitation']

In [ ]:
# This cell creates a dataframe where we have the following information per observation:
# all monthly values for each parameter during the corresponding year and season 
for i in range(len(parameters)):
    maize_2016_rabi = functions_model.time_observations(maize_2016_rabi, parameters[i], paths[i], statistic, methods[i])
    maize_2017_rabi = functions_model.time_observations(maize_2017_rabi, parameters[i], paths[i], statistic, methods[i])
    maize_2018_rabi = functions_model.time_observations(maize_2018_rabi, parameters[i], paths[i], statistic, methods[i])

    maize_2016_kharif = functions_model.time_observations(maize_2016_kharif, parameters[i], paths[i], statistic, methods[i])
    maize_2017_kharif = functions_model.time_observations(maize_2017_kharif, parameters[i], paths[i], statistic, methods[i])
    maize_2018_kharif = functions_model.time_observations(maize_2018_kharif, parameters[i], paths[i], statistic, methods[i])
    print(parameters[i] + ' added.')
maize_rabi = pd.concat([maize_2016_rabi, maize_2017_rabi, maize_2018_rabi], ignore_index=True)
maize_kharif = pd.concat([maize_2016_kharif, maize_2017_kharif, maize_2018_kharif], ignore_index=True)
# Time: 20 min 

In [16]:
# Save these dataframes to a geoJSON
maize_rabi.to_file(r'D:\other_thesis\maize_rabi_false.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
maize_kharif.to_file(r'D:\other_thesis\maize_kharif_false.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [17]:
# paths = ['D:/monthly_images/crop/NDVI/', 'D:/monthly_images/crop/EVI/', 'D:/monthly_images/crop/FAPAR/', 'D:/monthly_images/crop/LAI/', 'D:/monthly_images/crop/SIF/', 'D:/monthly_images/crop/SSM/', 'D:/monthly_images/crop/SUSM/', 'D:/monthly_images/crop/LST/', 'D:/monthly_images/crop/LSTN/', 'D:/monthly_images/crop/Precipitation/']
# methods = [False, False, False, False, True, True, True, True, True, True]
# statistic = ['median']
# parameters = ['NDVI', 'EVI', 'FAPAR', 'LAI', 'SIF', 'SSM', 'SUSM', 'LST', 'LSTN', 'Precipitation']

# maize_2016_rabi = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Rabi')] # 241
# maize_2016_zaid = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Summer')] # 23
# maize_2016_kharif = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Kharif')] # 1916
# maize_2017_rabi = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Rabi')] # 251
# maize_2017_zaid = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Summer')] # 16
# maize_2017_kharif = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Kharif')] # 1760
# maize_2018_rabi = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Rabi')] # 225
# maize_2018_zaid = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Summer')] # 8
# maize_2018_kharif = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Kharif')] # 1568

In [18]:
# for i in range(len(parameters)):
#     maize_2016_rabi = functions_model.time_observations(maize_2016_rabi, parameters[i], paths[i], statistic, methods[i])
#     maize_2017_rabi = functions_model.time_observations(maize_2017_rabi, parameters[i], paths[i], statistic, methods[i])
#     maize_2018_rabi = functions_model.time_observations(maize_2018_rabi, parameters[i], paths[i], statistic, methods[i])

#     maize_2016_kharif = functions_model.time_observations(maize_2016_kharif, parameters[i], paths[i], statistic, methods[i])
#     maize_2017_kharif = functions_model.time_observations(maize_2017_kharif, parameters[i], paths[i], statistic, methods[i])
#     maize_2018_kharif = functions_model.time_observations(maize_2018_kharif, parameters[i], paths[i], statistic, methods[i])
#     print(parameters[i] + ' added.')
# maize_rabi = pd.concat([maize_2016_rabi, maize_2017_rabi, maize_2018_rabi], ignore_index=True)
# maize_kharif = pd.concat([maize_2016_kharif, maize_2017_kharif, maize_2018_kharif], ignore_index=True)

NDVI added.
EVI added.
FAPAR added.
LAI added.
SIF added.
SSM added.
SUSM added.
LST added.
LSTN added.
Precipitation added.


In [19]:
# # True wrt LST and LSTN
# maize_rabi.to_file(r'D:\other_thesis\maize_rabi_true.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
# maize_kharif.to_file(r'D:\other_thesis\maize_kharif_true.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [20]:
# With some Falses
maize_kharif.isna().sum().sum() # 18931
maize_rabi.isna().sum().sum() # 195

np.size(maize_kharif.iloc[:,11:]) # 262200
print(18931/262200) # 0.07220061022120519 (percentage of missing values within the kharif season)
np.size(maize_rabi.iloc[:,11:]) # 57360
print(195/57360) # 0.003399581589958159 (percentage of missing values within the rabi season)

0.07220061022120519
0.003399581589958159


In [ ]:
# This cell investigates the NaN-values within the dataframe
maize_kharif.iloc[:,11:16] # NDVI
maize_kharif.iloc[:,16:21] # EVI
maize_kharif.iloc[:,21:26] # FAPAR
maize_kharif.iloc[:,26:31] # LAI
maize_kharif.iloc[:,31:36] # SIF
maize_kharif.iloc[:,36:41] # SSM
maize_kharif.iloc[:,41:46] # SUSM
maize_kharif.iloc[:,46:51] # LST
maize_kharif.iloc[:,51:56] # LSTN

maize_kharif.iloc[:,11:16][maize_kharif.iloc[:,11:16].isna().sum(axis=1) == 5] # NDVI, 3 (405, 2287, 4007), False
maize_kharif.iloc[:,16:21][maize_kharif.iloc[:,16:21].isna().sum(axis=1) == 5] # EVI, 3 (405, 2287, 4007), False
maize_kharif.iloc[:,21:26][maize_kharif.iloc[:,21:26].isna().sum(axis=1) == 5] # FAPAR 3 (405, 2287, 4007), False
maize_kharif.iloc[:,26:31][maize_kharif.iloc[:,26:31].isna().sum(axis=1) == 5] # LAI 3 (405, 2287, 4007), False
maize_kharif.iloc[:,31:36][maize_kharif.iloc[:,31:36].isna().sum(axis=1) == 5] # SIF 121 (among 405, 2287, 4007), True
maize_kharif.iloc[:,36:41][maize_kharif.iloc[:,36:41].isna().sum(axis=1) == 5] # SSM 190 (among 405, 2287, 4007), True
maize_kharif.iloc[:,41:46][maize_kharif.iloc[:,41:46].isna().sum(axis=1) == 5] # SUSM 190 (among 405, 2287, 4007), True
maize_kharif.iloc[:,46:51][maize_kharif.iloc[:,46:51].isna().sum(axis=1) == 5] # LST 19 (among 405, 2287, 4007), False (Deze zouden we eventueel nog kunnen switchen)
maize_kharif.iloc[:,51:56][maize_kharif.iloc[:,51:56].isna().sum(axis=1) == 5] # LSTN 19 (among 405, 2287, 4007), False (Deze zouden we eventueel nog kunnen switchen)

In [ ]:
# Save dataframe observations
maize_2016_rabi = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Rabi')] # 241
maize_2016_zaid = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Summer')] # 23
maize_2016_kharif = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Kharif')] # 1916
maize_2017_rabi = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Rabi')] # 251
maize_2017_zaid = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Summer')] # 16
maize_2017_kharif = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Kharif')] # 1760
maize_2018_rabi = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Rabi')] # 225
maize_2018_zaid = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Summer')] # 8
maize_2018_kharif = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Kharif')] # 1568

maize_2016_rabi.to_file(r'D:\other_thesis\maize_2016_rabi.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
maize_2016_zaid.to_file(r'D:\other_thesis\maize_2016_zaid.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
maize_2016_kharif.to_file(r'D:\other_thesis\maize_2016_kharif.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
maize_2017_rabi.to_file(r'D:\other_thesis\maize_2017_rabi.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
maize_2017_zaid.to_file(r'D:\other_thesis\maize_2017_zaid.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
maize_2017_kharif.to_file(r'D:\other_thesis\maize_2017_kharif.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
maize_2018_rabi.to_file(r'D:\other_thesis\maize_2018_rabi.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
maize_2018_zaid.to_file(r'D:\other_thesis\maize_2018_zaid.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
maize_2018_kharif.to_file(r'D:\other_thesis\maize_2018_kharif.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run

In [ ]:
# This cell creates a dataframe where we have the following information per observation:
# all daily values for Precipitation during the corresponding year and season 
maize_2016_rabi = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Rabi')] # 241
maize_2016_zaid = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Summer')] # 23
maize_2016_kharif = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Kharif')] # 1916
maize_2017_rabi = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Rabi')] # 251
maize_2017_zaid = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Summer')] # 16
maize_2017_kharif = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Kharif')] # 1760
maize_2018_rabi = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Rabi')] # 225
maize_2018_zaid = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Summer')] # 8
maize_2018_kharif = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Kharif')] # 1568

path_pre = 'D:/Data_download/GEE/Precipitation/Karnataka/cropmask/'

maize_2016_rabi = functions_model.time_observations_day(maize_2016_rabi, parameters[-1], path_pre, statistic, methods[-1]) # 4 min 
print('maize_2016_rabi done.')
maize_2017_rabi = functions_model.time_observations_day(maize_2017_rabi, parameters[-1], path_pre, statistic, methods[-1])
print('maize_2017_rabi done.')
maize_2018_rabi = functions_model.time_observations_day(maize_2018_rabi, parameters[-1], path_pre, statistic, methods[-1])
print('maize_2018_rabi done.')
maize_2016_kharif = functions_model.time_observations_day(maize_2016_kharif, parameters[-1], path_pre, statistic, methods[-1])
print('maize_2016_kharif done.')
maize_2017_kharif = functions_model.time_observations_day(maize_2017_kharif, parameters[-1], path_pre, statistic, methods[-1])
print('maize_2017_kharif done.')
maize_2018_kharif = functions_model.time_observations_day(maize_2018_kharif, parameters[-1], path_pre, statistic, methods[-1])
print('maize_2018_kharif done.')
maize_rabi_precipitation = pd.concat([maize_2016_rabi, maize_2017_rabi, maize_2018_rabi], ignore_index=True)
maize_kharif_precipitation = pd.concat([maize_2016_kharif, maize_2017_kharif, maize_2018_kharif], ignore_index=True)

In [24]:
# Save these dataframes
maize_rabi_precipitation.to_file(r'D:\other_thesis\maize_rabi_precipitation.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
maize_kharif_precipitation.to_file(r'D:\other_thesis\maize_kharif_precipitation.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [25]:
# Wel average voor: 
# LAI, (gemiddelde over drie images) OK
# NDVI, EVI (gemiddelde over twee images) OK
# SIF, (gemiddelde over twee images) OK
# FAPAR, (gemiddelde over drie images) OK 
# NIET OK:
# LST, LSTN, (gemiddelde over 30 images) Hier missen heel veel values, dus wellicht wel een goede approach hier
# SSM, SUSM (gemiddelde over 10 images)
# Voor precipitation de som gebruiken ipv average

In [29]:
# This cell creates a dataframe where we have the following information per observation:
# all monthly values for SSM during the corresponding year and season 
maize_2016_rabi = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Rabi')] # 241
maize_2016_zaid = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Summer')] # 23
maize_2016_kharif = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Kharif')] # 1916
maize_2017_rabi = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Rabi')] # 251
maize_2017_zaid = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Summer')] # 16
maize_2017_kharif = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Kharif')] # 1760
maize_2018_rabi = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Rabi')] # 225
maize_2018_zaid = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Summer')] # 8
maize_2018_kharif = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Kharif')] # 1568

path_pre = 'D:/Data_download/GEE/SSM/Karnataka/cropmask/'

maize_2016_rabi = functions_model.time_observations_day(maize_2016_rabi, 'SSM', path_pre, statistic, True) # 4 min 
print('maize_2016_rabi done.')
maize_2017_rabi = functions_model.time_observations_day(maize_2017_rabi, 'SSM', path_pre, statistic, True)
print('maize_2017_rabi done.')
maize_2018_rabi = functions_model.time_observations_day(maize_2018_rabi, 'SSM', path_pre, statistic, True)
print('maize_2018_rabi done.')
maize_2016_kharif = functions_model.time_observations_day(maize_2016_kharif, 'SSM', path_pre, statistic, True)
print('maize_2016_kharif done.')
maize_2017_kharif = functions_model.time_observations_day(maize_2017_kharif, 'SSM', path_pre, statistic, True)
print('maize_2017_kharif done.')
maize_2018_kharif = functions_model.time_observations_day(maize_2018_kharif, 'SSM', path_pre, statistic, True)
print('maize_2018_kharif done.')
maize_rabi_ssm = pd.concat([maize_2016_rabi, maize_2017_rabi, maize_2018_rabi], ignore_index=True)
maize_kharif_ssm = pd.concat([maize_2016_kharif, maize_2017_kharif, maize_2018_kharif], ignore_index=True)

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

maize_2016_rabi done.


c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

maize_2017_rabi done.


c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

maize_2018_rabi done.
maize_2016_kharif done.
maize_2017_kharif done.
maize_2018_kharif done.


In [30]:
# Save these dataframes
maize_rabi_ssm.to_file(r'D:\other_thesis\maize_rabi_ssm.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
maize_kharif_ssm.to_file(r'D:\other_thesis\maize_kharif_ssm.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [31]:
# This cell creates a dataframe where we have the following information per observation:
# all monthly values for SUSM during the corresponding year and season 
maize_2016_rabi = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Rabi')] # 241
maize_2016_zaid = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Summer')] # 23
maize_2016_kharif = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Kharif')] # 1916
maize_2017_rabi = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Rabi')] # 251
maize_2017_zaid = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Summer')] # 16
maize_2017_kharif = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Kharif')] # 1760
maize_2018_rabi = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Rabi')] # 225
maize_2018_zaid = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Summer')] # 8
maize_2018_kharif = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Kharif')] # 1568

path_pre = 'D:/Data_download/GEE/SUSM/Karnataka/cropmask/'

maize_2016_rabi = functions_model.time_observations_day(maize_2016_rabi, 'SUSM', path_pre, statistic, True) # 4 min 
print('maize_2016_rabi done.')
maize_2017_rabi = functions_model.time_observations_day(maize_2017_rabi, 'SUSM', path_pre, statistic, True)
print('maize_2017_rabi done.')
maize_2018_rabi = functions_model.time_observations_day(maize_2018_rabi, 'SUSM', path_pre, statistic, True)
print('maize_2018_rabi done.')
maize_2016_kharif = functions_model.time_observations_day(maize_2016_kharif, 'SUSM', path_pre, statistic, True)
print('maize_2016_kharif done.')
maize_2017_kharif = functions_model.time_observations_day(maize_2017_kharif, 'SUSM', path_pre, statistic, True)
print('maize_2017_kharif done.')
maize_2018_kharif = functions_model.time_observations_day(maize_2018_kharif, 'SUSM', path_pre, statistic, True)
print('maize_2018_kharif done.')
maize_rabi_susm = pd.concat([maize_2016_rabi, maize_2017_rabi, maize_2018_rabi], ignore_index=True)
maize_kharif_susm = pd.concat([maize_2016_kharif, maize_2017_kharif, maize_2018_kharif], ignore_index=True)

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

maize_2016_rabi done.


c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

maize_2017_rabi done.


c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

maize_2018_rabi done.
maize_2016_kharif done.
maize_2017_kharif done.
maize_2018_kharif done.


In [32]:
# Save these dataframes
maize_rabi_susm.to_file(r'D:\other_thesis\maize_rabi_susm.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
maize_kharif_susm.to_file(r'D:\other_thesis\maize_kharif_susm.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


The following cells are aggregating irr and rf yields such that we can visualize the average yields (kg/ha) as a map

In [120]:
yields_maize_2016_rabi = yields_maize[(yields_maize['Year'] == 2016) & (yields_maize['Season'] == 'Rabi')] # 241
yields_maize_2016_zaid = yields_maize[(yields_maize['Year'] == 2016) & (yields_maize['Season'] == 'Summer')] # 23
yields_maize_2016_kharif = yields_maize[(yields_maize['Year'] == 2016) & (yields_maize['Season'] == 'Kharif')] # 1916
yields_maize_2017_rabi = yields_maize[(yields_maize['Year'] == 2017) & (yields_maize['Season'] == 'Rabi')] # 251
yields_maize_2017_zaid = yields_maize[(yields_maize['Year'] == 2017) & (yields_maize['Season'] == 'Summer')] # 16
yields_maize_2017_kharif = yields_maize[(yields_maize['Year'] == 2017) & (yields_maize['Season'] == 'Kharif')] # 1760
yields_maize_2018_rabi = yields_maize[(yields_maize['Year'] == 2018) & (yields_maize['Season'] == 'Rabi')] # 225
yields_maize_2018_zaid = yields_maize[(yields_maize['Year'] == 2018) & (yields_maize['Season'] == 'Summer')] # 8
yields_maize_2018_kharif = yields_maize[(yields_maize['Year'] == 2018) & (yields_maize['Season'] == 'Kharif')] # 1568

yields_maize_2016_rabi_agg = yields_maize_2016_rabi.groupby(['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])['Average Yield(Kg/Ha)'].agg('sum').reset_index()
yields_maize_2016_zaid_agg = yields_maize_2016_zaid.groupby(['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])['Average Yield(Kg/Ha)'].agg('sum').reset_index()
yields_maize_2016_kharif_agg = yields_maize_2016_kharif.groupby(['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])['Average Yield(Kg/Ha)'].agg('sum').reset_index()
yields_maize_2017_rabi_agg = yields_maize_2017_rabi.groupby(['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])['Average Yield(Kg/Ha)'].agg('sum').reset_index()
yields_maize_2017_zaid_agg = yields_maize_2017_zaid.groupby(['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])['Average Yield(Kg/Ha)'].agg('sum').reset_index()
yields_maize_2017_kharif_agg = yields_maize_2017_kharif.groupby(['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])['Average Yield(Kg/Ha)'].agg('sum').reset_index()
yields_maize_2018_rabi_agg = yields_maize_2018_rabi.groupby(['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])['Average Yield(Kg/Ha)'].agg('sum').reset_index()
yields_maize_2018_zaid_agg = yields_maize_2018_zaid.groupby(['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])['Average Yield(Kg/Ha)'].agg('sum').reset_index()
yields_maize_2018_kharif_agg = yields_maize_2018_kharif.groupby(['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])['Average Yield(Kg/Ha)'].agg('sum').reset_index()

In [121]:
yields_maize_2016_rabi_agg = yields_maize_2016_rabi_agg.merge(yields_maize_2016_rabi, how = 'inner', on = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])
yields_maize_2016_rabi_agg = yields_maize_2016_rabi_agg.iloc[:,[0,1,2,3,4,5,6,11]]
yields_maize_2016_rabi_agg = yields_maize_2016_rabi_agg.rename(columns = {'Average Yield(Kg/Ha)_x': 'Average Yield(Kg/Ha)', 'geometry_x': 'geometry'})
yields_maize_2016_rabi_agg  = gpd.GeoDataFrame(yields_maize_2016_rabi_agg, geometry = yields_maize_2016_rabi_agg.geometry, crs = {'init': 'epsg:4326'})
yields_maize_2016_rabi_agg = yields_maize_2016_rabi_agg.drop_duplicates(ignore_index=True)#subset = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk', ''], ignore_index=True)

yields_maize_2016_kharif_agg = yields_maize_2016_kharif_agg.merge(yields_maize_2016_kharif, how = 'inner', on = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])
yields_maize_2016_kharif_agg = yields_maize_2016_kharif_agg.iloc[:,[0,1,2,3,4,5,6,11]]
yields_maize_2016_kharif_agg = yields_maize_2016_kharif_agg.rename(columns = {'Average Yield(Kg/Ha)_x': 'Average Yield(Kg/Ha)', 'geometry_x': 'geometry'})
yields_maize_2016_kharif_agg  = gpd.GeoDataFrame(yields_maize_2016_kharif_agg, geometry = yields_maize_2016_kharif_agg.geometry, crs = {'init': 'epsg:4326'})
yields_maize_2016_kharif_agg = yields_maize_2016_kharif_agg.drop_duplicates(ignore_index=True)#subset = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk', ''], ignore_index=True)

yields_maize_2016_zaid_agg = yields_maize_2016_zaid_agg.merge(yields_maize_2016_zaid, how = 'inner', on = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])
yields_maize_2016_zaid_agg = yields_maize_2016_zaid_agg.iloc[:,[0,1,2,3,4,5,6,11]]
yields_maize_2016_zaid_agg = yields_maize_2016_zaid_agg.rename(columns = {'Average Yield(Kg/Ha)_x': 'Average Yield(Kg/Ha)', 'geometry_x': 'geometry'})
yields_maize_2016_zaid_agg  = gpd.GeoDataFrame(yields_maize_2016_zaid_agg, geometry = yields_maize_2016_zaid_agg.geometry, crs = {'init': 'epsg:4326'})
yields_maize_2016_zaid_agg = yields_maize_2016_zaid_agg.drop_duplicates(ignore_index=True)#subset = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk', ''], ignore_index=True)
#

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred i

In [122]:
yields_maize_2017_rabi_agg = yields_maize_2017_rabi_agg.merge(yields_maize_2017_rabi, how = 'inner', on = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])
yields_maize_2017_rabi_agg = yields_maize_2017_rabi_agg.iloc[:,[0,1,2,3,4,5,6,11]]
yields_maize_2017_rabi_agg = yields_maize_2017_rabi_agg.rename(columns = {'Average Yield(Kg/Ha)_x': 'Average Yield(Kg/Ha)', 'geometry_x': 'geometry'})
yields_maize_2017_rabi_agg  = gpd.GeoDataFrame(yields_maize_2017_rabi_agg , geometry = yields_maize_2017_rabi_agg.geometry, crs = {'init': 'epsg:4326'})
yields_maize_2017_rabi_agg = yields_maize_2017_rabi_agg.drop_duplicates(ignore_index=True)#subset = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk', ''], ignore_index=True)

yields_maize_2017_kharif_agg = yields_maize_2017_kharif_agg.merge(yields_maize_2017_kharif, how = 'inner', on = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])
yields_maize_2017_kharif_agg = yields_maize_2017_kharif_agg.iloc[:,[0,1,2,3,4,5,6,11]]
yields_maize_2017_kharif_agg = yields_maize_2017_kharif_agg.rename(columns = {'Average Yield(Kg/Ha)_x': 'Average Yield(Kg/Ha)', 'geometry_x': 'geometry'})
yields_maize_2017_kharif_agg  = gpd.GeoDataFrame(yields_maize_2017_kharif_agg, geometry = yields_maize_2017_kharif_agg.geometry, crs = {'init': 'epsg:4326'})
yields_maize_2017_kharif_agg = yields_maize_2017_kharif_agg.drop_duplicates(ignore_index=True)#subset = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk', ''], ignore_index=True)

yields_maize_2017_zaid_agg = yields_maize_2017_zaid_agg.merge(yields_maize_2017_zaid, how = 'inner', on = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])
yields_maize_2017_zaid_agg = yields_maize_2017_zaid_agg.iloc[:,[0,1,2,3,4,5,6,11]]
yields_maize_2017_zaid_agg = yields_maize_2017_zaid_agg.rename(columns = {'Average Yield(Kg/Ha)_x': 'Average Yield(Kg/Ha)', 'geometry_x': 'geometry'})
yields_maize_2017_zaid_agg  = gpd.GeoDataFrame(yields_maize_2017_zaid_agg , geometry = yields_maize_2017_zaid_agg.geometry, crs = {'init': 'epsg:4326'})
yields_maize_2017_zaid_agg = yields_maize_2017_zaid_agg.drop_duplicates(ignore_index=True)#subset = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk', ''], ignore_index=True)
#

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred i

In [123]:
yields_maize_2018_rabi_agg = yields_maize_2018_rabi_agg.merge(yields_maize_2018_rabi, how = 'inner', on = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])
yields_maize_2018_rabi_agg = yields_maize_2018_rabi_agg.iloc[:,[0,1,2,3,4,5,6,11]]
yields_maize_2018_rabi_agg = yields_maize_2018_rabi_agg.rename(columns = {'Average Yield(Kg/Ha)_x': 'Average Yield(Kg/Ha)', 'geometry_x': 'geometry'})
yields_maize_2018_rabi_agg  = gpd.GeoDataFrame(yields_maize_2018_rabi_agg , geometry = yields_maize_2018_rabi_agg.geometry, crs = {'init': 'epsg:4326'})
yields_maize_2018_rabi_agg = yields_maize_2018_rabi_agg.drop_duplicates(ignore_index=True)#subset = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk', ''], ignore_index=True)

yields_maize_2018_kharif_agg = yields_maize_2018_kharif_agg.merge(yields_maize_2018_kharif, how = 'inner', on = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])
yields_maize_2018_kharif_agg = yields_maize_2018_kharif_agg.iloc[:,[0,1,2,3,4,5,6,11]]
yields_maize_2018_kharif_agg = yields_maize_2018_kharif_agg.rename(columns = {'Average Yield(Kg/Ha)_x': 'Average Yield(Kg/Ha)', 'geometry_x': 'geometry'})
yields_maize_2018_kharif_agg  = gpd.GeoDataFrame(yields_maize_2018_kharif_agg , geometry = yields_maize_2018_kharif_agg.geometry, crs = {'init': 'epsg:4326'})
yields_maize_2018_kharif_agg = yields_maize_2018_kharif_agg.drop_duplicates(ignore_index=True)#subset = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk', ''], ignore_index=True)

yields_maize_2018_zaid_agg = yields_maize_2018_zaid_agg.merge(yields_maize_2018_zaid, how = 'inner', on = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk'])
yields_maize_2018_zaid_agg = yields_maize_2018_zaid_agg.iloc[:,[0,1,2,3,4,5,6,11]]
yields_maize_2018_zaid_agg = yields_maize_2018_zaid_agg.rename(columns = {'Average Yield(Kg/Ha)_x': 'Average Yield(Kg/Ha)', 'geometry_x': 'geometry'})
yields_maize_2018_zaid_agg  = gpd.GeoDataFrame(yields_maize_2018_zaid_agg , geometry = yields_maize_2018_zaid_agg.geometry, crs = {'init': 'epsg:4326'})
yields_maize_2018_zaid_agg = yields_maize_2018_zaid_agg.drop_duplicates(ignore_index=True)#subset = ['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk', ''], ignore_index=True)
#

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred i

In [124]:
yields_maize_2016_rabi_agg.to_file(r'D:\other_thesis\yields_maize_2016_rabi_agg.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
yields_maize_2016_zaid_agg.to_file(r'D:\other_thesis\yields_maize_2016_zaid_agg.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
yields_maize_2016_kharif_agg.to_file(r'D:\other_thesis\yields_maize_2016_kharif_agg.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
yields_maize_2017_rabi_agg.to_file(r'D:\other_thesis\yields_maize_2017_rabi_agg.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
yields_maize_2017_zaid_agg.to_file(r'D:\other_thesis\yields_maize_2017_zaid_agg.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
yields_maize_2017_kharif_agg.to_file(r'D:\other_thesis\yields_maize_2017_kharif_agg.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
yields_maize_2018_rabi_agg.to_file(r'D:\other_thesis\yields_maize_2018_rabi_agg.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
yields_maize_2018_zaid_agg.to_file(r'D:\other_thesis\yields_maize_2018_zaid_agg.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
yields_maize_2018_kharif_agg.to_file(r'D:\other_thesis\yields_maize_2018_kharif_agg.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype in

In [138]:
def soiltype(geodataframe, parameter, path, statistic, method):
    # action = True/False
    gdf = geodataframe.copy()
    year = geodataframe['Year'].unique()[0] # Year
    season = geodataframe['Season'].unique()[0].lower() # Season

    file_name = 'soiltype_Karnataka_' + str(year) + '.tif'

    file_path = path + file_name
    param = rasterio.open(file_path, mode = 'r')
    # Assign raster values to a numpy nd array
    param_array = param.read(1) # landuse corresponding to each rasterpixel, so we extracted the pixel values from the raster    
    # NOTE: stats must be any of these ['count', 'min', 'max', 'mean', 'sum', 'std', 'median', 'majority', 'minority', 'unique', 'range', 'nodata', 'nan']
    zonal_stat = zonal_stats(geodataframe.geometry, param_array, affine = param.transform, geojson_out = True, nodata = param.nodata, all_touched = method, stats = statistic)
    
    # Extracting the statistics from the list
    for j in range(len(statistic)):
        gdf[parameter + '_' + statistic[j]] = np.nan
        indx = list(gdf.columns).index(parameter + '_' + statistic[j])
        for i in range(np.size(geodataframe,0)):
            gdf.iloc[i,indx] = zonal_stat[i]['properties'][statistic[j]]

    return gdf
# Nu alleen nog de metrics toevoegen: slope en gaan we al afnemen?

In [139]:
maize_2016_rabi = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Rabi')] # 241
maize_2016_zaid = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Summer')] # 23
maize_2016_kharif = obs_maize[(obs_maize['Year'] == 2016) & (obs_maize['Season'] == 'Kharif')] # 1916
maize_2017_rabi = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Rabi')] # 251
maize_2017_zaid = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Summer')] # 16
maize_2017_kharif = obs_maize[(obs_maize['Year'] == 2017) & (obs_maize['Season'] == 'Kharif')] # 1760
maize_2018_rabi = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Rabi')] # 225
maize_2018_zaid = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Summer')] # 8
maize_2018_kharif = obs_maize[(obs_maize['Year'] == 2018) & (obs_maize['Season'] == 'Kharif')] # 1568

maize_2016_rabi = soiltype(maize_2016_rabi, 'Soiltype', 'D:/Data_download/Other/Soiltype/cropmask/', ['majority'], False)
maize_2016_zaid = soiltype(maize_2016_zaid, 'Soiltype', 'D:/Data_download/Other/Soiltype/cropmask/', ['majority'], False)
maize_2016_kharif = soiltype(maize_2016_kharif, 'Soiltype', 'D:/Data_download/Other/Soiltype/cropmask/', ['majority'], False)
maize_2017_rabi = soiltype(maize_2017_rabi, 'Soiltype', 'D:/Data_download/Other/Soiltype/cropmask/', ['majority'], False)
maize_2017_zaid = soiltype(maize_2017_zaid, 'Soiltype', 'D:/Data_download/Other/Soiltype/cropmask/', ['majority'], False)
maize_2017_kharif = soiltype(maize_2017_kharif, 'Soiltype', 'D:/Data_download/Other/Soiltype/cropmask/', ['majority'], False)
maize_2018_rabi = soiltype(maize_2018_rabi, 'Soiltype', 'D:/Data_download/Other/Soiltype/cropmask/', ['majority'], False)
maize_2018_zaid = soiltype(maize_2018_zaid, 'Soiltype', 'D:/Data_download/Other/Soiltype/cropmask/', ['majority'], False)
maize_2018_kharif = soiltype(maize_2018_kharif, 'Soiltype', 'D:/Data_download/Other/Soiltype/cropmask/', ['majority'], False)


maize_rabi = pd.concat([maize_2016_rabi, maize_2017_rabi, maize_2018_rabi], ignore_index=True)
maize_zaid = pd.concat([maize_2016_zaid, maize_2017_zaid, maize_2018_zaid], ignore_index=True)
maize_kharif = pd.concat([maize_2016_kharif, maize_2017_kharif, maize_2018_kharif], ignore_index=True)

maize_rabi.to_file(r'D:\other_thesis\maize_rabi_soiltype.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
maize_zaid.to_file(r'D:\other_thesis\maize_zaid_soiltype.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run
maize_kharif.to_file(r'D:\other_thesis\maize_kharif_soiltype.geojson', driver="GeoJSON") # Save the derived datframe as it takes a long time to run

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
